In [1]:
!pip install gurobipy
import numpy as np
from gurobipy import *

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 11.5 MB 16.2 MB/s 


Khởi tạo mô hình và trọng số

In [2]:
m = Model()
u = np.array([
    [0, 3, 7, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 9, 3, 0, 0, 0, 0],
    [0, 0, 0, 0, 3, 4, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 1, 0, 0],
    [0, 0, 0, 0, 0, 0, 5, 8, 0],
    [0, 0, 0, 0, 0, 0, 0, 5, 0],
    [0, 0, 0, 0, 0, 0, 0, 0, 8],
    [0, 0, 0, 0, 0, 0, 0, 0, 3],
    [0, 0, 0, 0, 0, 0, 0, 0, 0]
])
d_vertex = len(u)

Restricted license - for non-production use only - expires 2023-10-25


Khai báo biến

In [3]:
V = [i for i in range(d_vertex)]
E = [(i, j) for i in V for j in V if u[i, j] != 0]

a = m.addVars(V, vtype = GRB.CONTINUOUS, lb = -GRB.INFINITY, name = 'a')
g = m.addVars(E, vtype = GRB.CONTINUOUS, lb = -GRB.INFINITY, name = 'g')
b = m.addVars(E, vtype = GRB.BINARY, name = 'b')
x = m.addVars(E, vtype = GRB.BINARY, name = 'x')

Hàm mục tiêu

In [4]:
m.setObjective(quicksum(u[i, j]*(b[i, j] - g[i, j]) for (i, j) in E), GRB.MINIMIZE)

Khai báo điều kiện ràng buộc

In [5]:
m.addConstrs(g[i, j] <= x[i, j] for (i, j) in E)
m.addConstrs(g[i, j] <= b[i, j] for (i, j) in E)
m.addConstrs(a[i] - a[j] + b[i, j] >= 0 for (i, j) in E)
m.addConstr(a[8] - a[0] >= 1)
m.addConstr(quicksum(x[i, j] for (i, j) in E) <= 1)
m.update()

In [6]:
m.optimize()

Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (linux64)
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads
Optimize a model with 38 rows, 45 columns and 98 nonzeros
Model fingerprint: 0x7497be51
Variable types: 21 continuous, 24 integer (24 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 9e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective 11.0000000
Presolve removed 19 rows and 20 columns
Presolve time: 0.00s
Presolved: 19 rows, 25 columns, 58 nonzeros
Variable types: 0 continuous, 25 integer (24 binary)
Found heuristic solution: objective 3.0000000

Root relaxation: cutoff, 8 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0     cutoff    0         3.00000    3.00000  0.00%     -    0s

Explored 1 node

In [7]:
m.printAttr('X')


    Variable            X 
-------------------------
        a[8]            1 
      g[6,8]            1 
      b[6,8]            1 
      b[7,8]            1 
      x[6,8]            1 
